Langkah 1 Import Library

In [ ]:
import pandas as pd
from transformers import  AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split

Langkah 2 Masukan dataset dan cek dataset, serta merubah string ke integer pada kolom labeling

In [ ]:
# df_labeled = pd.read_csv('ada labelnya - Sheet1.csv')
# df_unlabeled = pd.read_csv('gaada labelnya - Sheet1.csv')

# df_unlabeled = df_unlabeled.rename(columns={"Data Komentar TIKTOK": "text"})

# df_labeled["Labeling"] = df_labeled["Labeling"].str.lower().map({"netral": 0, "negatif": 1, "positif": 2})

# print("Dataset Labeled:")
# df_labeled.head()

df_train = pd.read_csv("ada labelnya - Sheet1.csv")
df_train = df_train.rename(columns={"Data Komentar TIKTOK": "text", "Labeling": "label"})
label_map = {"negatif": 0, "netral": 1, "positif": 2}
df_train = df_train[df_train["label"].isin(label_map.keys())]
df_train["label"] = df_train["label"].map(label_map)

In [ ]:
# print("Dataset Unlabeled:")
# df_unlabeled.head()

df_test = pd.read_csv("gaada labelnya - Sheet1.csv")
df_test = df_test.rename(columns={"Data Komentar TIKTOK": "text"})

Langkah 3 Load model sentimen IndoRoBERTa yang sudah di-finetune

In [33]:
model_name = "cahya/roberta-base-indonesian-522M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

/Users/habibfauzanmahardika/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cahya/roberta-base-indonesian-522M were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenc

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

Langkah 4 Mapping label

In [34]:
label_map = {0: "netral", 1: "negatif", 2: "positif"}
text_column = "Labeling"

Langkah 5 Membuat fungsi prediksi

In [35]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_label = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_label].item()
    return label_map[pred_label], confidence

Langkah 6 Prediksi seluruh dataset

In [36]:
sentiments = []
confidences = []

for text in tqdm(df_unlabeled["text"]):
    label, conf = predict_sentiment(str(text))
    sentiments.append(label)
    confidences.append(conf)

100%|██████████| 300/300 [00:09<00:00, 32.11it/s]


Langkah 7 Tambahkan hasil ke dataframe

In [37]:
df_unlabeled["predicted_sentiment"] = sentiments
df_unlabeled["confidence"] = confidences

Langkah 8 Simpan hasil sentimen ke file excel baru

In [38]:
output_path = "hasil_sentimen_unlabeled.csv"
df_unlabeled.to_csv(output_path, index=False)

print(f"Hasil disimpan di: {output_path}")

Hasil disimpan di: hasil_sentimen_unlabeled.csv
